In [1]:
!pip install datasets
!pip install openpyxl
!pip install -q -U google-genai
!pwd

/Users/oribar-joseph/PycharmProjects/qwen-hebrew-finetuning1/synth


In [2]:
from datasets import load_dataset
import random
!pip install google-generativeai
import google.generativeai as genai
import time
import pandas as pd

/Users/oribar-joseph/PycharmProjects/qwen-hebrew-finetuning1/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import google.generativeai as genai

In [4]:


ds = load_dataset("nvidia/OpenMathReasoning", split="cot", streaming=True)

In [5]:
# Block 2: Fast diversity sampling function
def get_diverse_examples_fast(ds, n_samples=5, total_samples=5000):
    """Fast method using reservoir sampling to get examples with diverse COT lengths"""

    # Initialize buckets for different length ranges
    buckets = {
        'short': [],  # 0-500 chars
        'medium': [],  # 500-1500 chars
        'long': [],  # 1500+ chars
    }

    count = 0
    for item in ds:
        if count >= total_samples:
            break

        cot_length = len(item["generated_solution"])

        # Determine bucket
        if cot_length < 500:
            bucket_name = 'short'
        elif cot_length < 1500:
            bucket_name = 'medium'
        else:
            bucket_name = 'long'

        # Add to bucket (keep max 100 per bucket)
        bucket = buckets[bucket_name]
        if len(bucket) < 100:
            bucket.append((item["problem"], item["generated_solution"], item["expected_answer"]))
        else:
            # Replace random item (reservoir sampling)
            replace_idx = random.randint(0, len(bucket))
            if replace_idx < len(bucket):
                bucket[replace_idx] = (item["problem"], item["generated_solution"], item["expected_answer"])

        count += 1

    # Select examples from buckets to get diversity
    diverse_examples = []
    for bucket_name in ['short', 'medium', 'long']:
        if buckets[bucket_name]:
            # Take multiple from each bucket if needed
            samples_from_bucket = min(len(buckets[bucket_name]), max(1, n_samples // 3))
            diverse_examples.extend(random.sample(buckets[bucket_name], samples_from_bucket))

    # If we need more samples, fill from any bucket
    while len(diverse_examples) < n_samples:
        for bucket in buckets.values():
            if bucket and len(diverse_examples) < n_samples:
                remaining = [ex for ex in bucket if ex not in diverse_examples]
                if remaining:
                    diverse_examples.append(random.choice(remaining))

    return diverse_examples[:n_samples]

In [6]:
# Pair them together
# Block 3: Get diverse examples (this replaces your slow sorting)
examples = get_diverse_examples_fast(ds, n_samples=5, total_samples=5000)

# Block 4: Check results
print(f"Selected {len(examples)} diverse examples:")
for i, (q, cot, ans) in enumerate(examples):
    print(f"Example {i + 1}: COT length = {len(cot)} chars")

# Block 5: Extract fields if you need them separately
questions = [ex[0] for ex in examples]
cot_solutions = [ex[1] for ex in examples]
expected_answers = [ex[2] for ex in examples]

Selected 5 diverse examples:
Example 1: COT length = 1278 chars
Example 2: COT length = 25241 chars
Example 3: COT length = 29271 chars
Example 4: COT length = 50501 chars
Example 5: COT length = 45700 chars


In [7]:
GEMINI_API_KEY = ""  # Replace with your actual API key
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel('gemini-2.5-pro')

In [8]:
def translate_to_hebrew(text, text_type="text"):
    """Translate text to Hebrew using Gemini API with rate limiting"""
    try:
        if text_type == "math_problem":
            prompt = f"Translate the following mathematical problem to Hebrew. Keep all mathematical expressions, numbers, and formulas unchanged. Only translate the text portions:\n\n{text}"
        elif text_type == "solution":
            prompt = f"Translate the following mathematical solution to Hebrew. Keep all mathematical expressions, calculations, and formulas unchanged. Only translate the explanatory text:\n\n{text}"
        else:
            prompt = f"Translate the following to Hebrew:\n\n{text}"

        response = model.generate_content(prompt)
        time.sleep(0.5)  # Rate limiting - adjust as needed
        return response.text
    except Exception as e:
        print(f"Translation error: {e}")
        return text

In [9]:
# Translate all content
print("Translating questions...")
translated_questions = []
for i, question in enumerate(questions):
    print(f"Translating question {i+1}/{len(questions)}")
    translated_questions.append(translate_to_hebrew(question, "math_problem"))

print("Translating COT solutions...")
translated_cot_solutions = []
for i, cot in enumerate(cot_solutions):
    print(f"Translating solution {i+1}/{len(cot_solutions)}")
    translated_cot_solutions.append(translate_to_hebrew(cot, "solution"))

print("Translating answers...")
translated_answers = []
for i, answer in enumerate(expected_answers):
    print(f"Translating answer {i+1}/{len(expected_answers)}")
    translated_answers.append(translate_to_hebrew(answer))

print("Translation complete!")

Translating questions...
Translating question 1/5
Translating question 2/5
Translating question 3/5
Translating question 4/5
Translating question 5/5
Translating COT solutions...
Translating solution 1/5
Translating solution 2/5
Translating solution 3/5
Translating solution 4/5
Translating solution 5/5
Translating answers...
Translating answer 1/5
Translating answer 2/5
Translating answer 3/5
Translating answer 4/5
Translating answer 5/5
Translation complete!


In [10]:
def create_few_shot_prompt_for_hebrew_cot(english_questions, translated_cot_solutions, english_answers, new_english_question):
    """Create a few-shot prompt to generate Hebrew CoT for English questions"""

    prompt = """You are an expert mathematics tutor who provides detailed step-by-step solutions in Hebrew. Given mathematical problems in English, you explain the solution process in fluent Hebrew while keeping all mathematical expressions and formulas in their original form.

Your Hebrew explanations should be:
- Comprehensive and show all reasoning steps
- Clear and easy to follow
- Use proper Hebrew mathematical terminology
- Keep numbers, equations, and mathematical symbols unchanged

Here are examples of how to solve English mathematical problems with detailed Hebrew explanations:

"""

    # Add few-shot examples (English question -> Hebrew CoT -> English answer)
    for i in range(len(english_questions)):
        prompt += f"""Example {i+1}:
English Question: {english_questions[i]}

Hebrew Solution:
{translated_cot_solutions[i]}

Final Answer: {english_answers[i]}

---

"""

    # Add the new question to solve
    prompt += f"""Now solve this problem following the same pattern - provide a detailed Hebrew explanation:

English Question: {new_english_question}

Hebrew Solution:"""

    return prompt

In [11]:
def generate_hebrew_cot_for_dataset(english_questions, translated_cot_solutions, english_answers,
                                  num_questions=10, model=model):
    """
    Generate Hebrew CoT for questions from the original dataset

    Args:
        english_questions: List of English questions (few-shot examples)
        translated_cot_solutions: List of Hebrew translated CoT solutions (few-shot examples)
        english_answers: List of English answers (few-shot examples)
        num_questions: Number of questions to process from dataset
        model: Gemini model instance

    Returns:
        pandas DataFrame with columns: question_english, generated_cot_hebrew, real_cot_english, real_answer_english
    """

    # Load the dataset
    ds = load_dataset("nvidia/OpenMathReasoning", split="cot", streaming=True)

    results = []
    count = 0

    print(f"Processing {num_questions} questions from the dataset...")

    for item in ds:
        if count >= num_questions:
            break

        try:
            # Extract data from dataset
            english_question = item["problem"]
            real_english_cot = item["generated_solution"]
            real_english_answer = item["expected_answer"]

            print(f"Processing question {count+1}/{num_questions}")
            print(f"Question preview: {english_question[:100]}...")

            # Create few-shot prompt
            prompt = create_few_shot_prompt_for_hebrew_cot(
                english_questions,
                translated_cot_solutions,
                english_answers,
                english_question
            )

            # Generate Hebrew CoT
            response = model.generate_content(prompt)
            generated_hebrew_cot = response.text

            # Add to results
            result = {
                'question_english': english_question,
                'generated_cot_hebrew': generated_hebrew_cot,
                'real_cot_english': real_english_cot,
                'real_answer_english': real_english_answer
            }

            results.append(result)
            print(f"✓ Successfully generated Hebrew CoT for question {count+1}")

            # Rate limiting
            time.sleep(1)  # Adjust as needed for API rate limits

        except Exception as e:
            print(f"✗ Error processing question {count+1}: {e}")
            # Add error result to maintain count
            result = {
                'question_english': item.get("problem", "Error loading question"),
                'generated_cot_hebrew': f"Error: {str(e)}",
                'real_cot_english': item.get("generated_solution", "Error loading CoT"),
                'real_answer_english': item.get("expected_answer", "Error loading answer")
            }
            results.append(result)

        count += 1

    # Convert to DataFrame
    df = pd.DataFrame(results)
    return df

In [12]:
hebrew_cot_df = generate_hebrew_cot_for_dataset(
    english_questions=questions,  # Your original English questions
    translated_cot_solutions=translated_cot_solutions,  # Hebrew CoT solutions
    english_answers=expected_answers,  # Your original English answers
    num_questions=10,
    model=model
)

Processing 10 questions from the dataset...
Processing question 1/10
Question preview: Given a group of \( N \) balls consisting of \( C \) colors, where the number of balls in each color...
✓ Successfully generated Hebrew CoT for question 1
Processing question 2/10
Question preview: How many lines can be drawn that are equidistant from \( n \) given points in the plane?...
✓ Successfully generated Hebrew CoT for question 2
Processing question 3/10
Question preview: Find all functions \( f: \mathbb{R} \to \mathbb{R} \) such that
\[
(f(x^2) + f(xy) + f(y^2))(x - y) ...
✓ Successfully generated Hebrew CoT for question 3
Processing question 4/10
Question preview: Find the sum of the roots of the equation \((x+6)^{1/3} + (x-2)^{1/2} = 2\)....
✓ Successfully generated Hebrew CoT for question 4
Processing question 5/10
Question preview: Determine how many 1000 digit numbers \( A \) have the property that when any digit of \( A \), asid...
✓ Successfully generated Hebrew CoT for question 5
Pr

In [13]:
hebrew_cot_df

,question_english,generated_cot_hebrew,real_cot_english,real_answer_english
0,Given a group of \( N \) balls consisting of \...,כדי לחשב את ההסתברות שהכדורים שנבחרו יכללו בדי...,"<think>\nOkay, so I need to find the probabili...",\(\frac{C_{n_1}^{a_1} \cdot C_{n_2}^{a_2} \cdo...
1,How many lines can be drawn that are equidista...,"כדי לענות על השאלה ""כמה ישרים ניתן להעביר הנמצ...","<think>\nOkay, so the problem is asking how ma...",\frac{n(n-1)}{2}
2,Find all functions \( f: \mathbb{R} \to \mathb...,כדי למצוא את כל הפונקציות \( f: \mathbb{R} \to...,"<think>\nOkay, let's try to solve this functio...",\( f(x) = f(1)x \)
3,Find the sum of the roots of the equation \((x...,כדי למצוא את סכום השורשים של המשוואה \((x+6)^{...,"<think>\nOkay, let's see. I need to find the s...",2
4,Determine how many 1000 digit numbers \( A \) ...,כדי לקבוע כמה מספרים בני 1000 ספרות \( A \) מק...,"<think>\nOkay, so I need to figure out how man...",32
5,Calculate the integral\n\n\[\n\int^{\frac{3\pi...,כדי לחשב את האינטגרל\n\[\n\int^{\frac{3\pi}{2}...,"<think>\nOkay, let me try to solve this integr...",\(\frac{2}{\pi} + \frac{32}{9\pi^2}\)
6,"In $\triangle ABC$ with incenter $I$, points $...","כדי לפתור את הבעיה, עלינו למצוא את היחס \(\fra...","<think>\nAlright, let me try to tackle this ge...",1
7,"Let \( x_0 = 1 \), and \( x_{n+1} = 2x_n + \sq...",Hebrew Solution:\nכדי לקבוע את ערכו של \( x_{2...,"<think>\nOkay, let's see. The problem is about...",\(\frac{(3-\sqrt{3})(2-\sqrt{3})^{2016}+(3+\sq...
8,"Which is greater, $\sqrt[2012]{2013!}$ or $\sq...","כדי לקבוע איזה משני הביטויים גדול יותר, \(\sqr...","<think>\nOkay, so I need to figure out whether...",$\sqrt[2012]{2013!} > \sqrt[2013]{2012!}$
9,"On average, how long will you have to flip a c...",כדי לחשב את תוחלת מספר ההטלות של מטבע הוגן עד ...,"<think>\nOkay, so I need to figure out the exp...",20


In [14]:
hebrew_cot_df.to_csv('/Users/oribar-joseph/Downloads/cot.csv')